In [1]:
import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import nibabel as nib
import matplotlib.pyplot as plt
from dataset import CarotidDataset
from unet import UNet

from torchvision import transforms

In [2]:
data = CarotidDataset()

epochs = 2
BATCH_SIZE = 4
net = UNet(in_channels=3, n_classes=1, padding=True, up_mode='upsample')
device = torch.device("mps")
#device = torch.device("cpu")
trainloader = torch.utils.data.DataLoader(data, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
step_count = 0
loss = nn.CrossEntropyLoss()

transforms = torch.nn.Sequential(
    transforms.CenterCrop((736, 704))
)

In [3]:
net.train()
net = net.to(device)
train_running_loss = 0.0
train_running_corrects = 0

In [4]:
inputs, labels = next(iter(trainloader))
inputs = inputs.to(device)
labels = labels.to(device)
optimizer.zero_grad()
preds = net(inputs)
preds = F.pad(preds, (3,2,7,6))

In [5]:
preds.shape

torch.Size([4, 1, 749, 709])

In [6]:
preds.sum(dim=[1,2,3])

tensor([131529.6094, 146300.2500, 135946.7188, 125810.6406], device='mps:0',
       grad_fn=<SumBackward1>)

In [7]:
labels.sum(dim=[1,2,3])

tensor([5613570, 3314745, 2944995, 2559180], device='mps:0')

In [13]:
mask = labels > 0

In [23]:
labels.max()

tensor(255, device='mps:0', dtype=torch.uint8)

In [24]:
mask.max()

tensor(True, device='mps:0')

In [ ]:
diceLabel = labels.sum(dim=[1,2,3])
dicePrediction = preds.sum(dim=[1,2,3])
diceCorrect = (preds * labels).sum(dim=[1,2,3])
epsilon = 1

In [ ]:
diceRatio = (2. * diceCorrect + epsilon)/(dicePrediction + diceLabel + epsilon)

In [ ]:
diceRatio

In [ ]:
1 - diceRatio

In [ ]:
preds.view(-1).shape

In [ ]:
labels.shape

In [ ]:
train_loss = dice_loss(preds,labels)

In [20]:
class DiceLoss(nn.Module):
    def __init__(self, sigmoid=True):
        super(DiceLoss, self).__init__()
        self.sigmoid = sigmoid

    def dice(self, preds, labels, epsilon):
        if self.sigmoid == True:
            preds = F.sigmoid(preds)
        labels = labels > 0    
            
        diceLabel = labels.sum(dim=[1,2,3])
        dicePrediction = preds.sum(dim=[1,2,3])
        diceCorrect = (preds * labels).sum(dim=[1,2,3])

        diceRatio = (2. * diceCorrect + epsilon)/(dicePrediction + diceLabel + epsilon)       
        return (1 - diceRatio).mean()

    def forward(self, preds, labels, epsilon=1):
        dice_loss = self.dice(preds, labels, epsilon)
        fn_loss = self.dice(preds * labels, labels, epsilon)
        return dice_loss + fn_loss * 8
dice_loss = DiceLoss()

In [21]:
train_loss = dice_loss(preds, labels)

In [22]:
train_loss

tensor(8.1671, device='mps:0', grad_fn=<AddBackward0>)